# E. Coli Met/Thermo Example
This is a detailed example on how to use pickaxe and the filters it comes with to:
1. Generate an expansion of E. Coli using Pickaxe Rules
2. Apply Metabolomics/Thermodynamics Filters
3. Produce Plots

In [9]:
from minedatabase import pickaxe, rules
from minedatabase.filters import MetabolomicsFilter, AtomicCompositionFilter
from minedatabase.filters.feasibility import _get_feasibility, ReactionFeasibilityFilter
from time import time
from collections import defaultdict
from IPython.display import clear_output
import pandas as pd
import seaborn as sns
import multiprocessing as mp
from math import ceil
import pickle

class StopWatch():
    def __init__(self):
        self._start = time()

    def start(self):
        self._start = time()

    def tick(self):
        return time() - self._start

# 1. Generate or Load Expansion

In [2]:
n_cores = 11
n_rules = 50
input_cpds = "../data/ecocyc_smiles_sanitized_600.csv"

timer = StopWatch()

In [4]:
pk = pickaxe.Pickaxe()
pk.load_pickled_pickaxe("../data/ecoli_metab_100t300r200.pk")

----------------------------------------
Intializing pickaxe object

Done intializing pickaxe object
----------------------------------------

Loading ../data/ecoli_metab_100t300r200.pk pickled data.
Loaded 21336 compounds
Loaded 38175 reactions
Loaded 50 operators
Took 0.6575591564178467


## Generate Data
Took ~1hr with 12 processes

In [7]:
timer.start()
rule_io, correactant_list, _ = rules.metacyc_generalized(n_rules=n_rules)

# Load Pickaxe
pk = pickaxe.Pickaxe(coreactant_list=correactant_list, rule_list=rule_io, filter_after_final_gen=True)
cpds = pk.load_compound_set(input_cpds)

# Set up Metabolomics Filter
metFilter = MetabolomicsFilter(
    filter_name="ADP1_Metabolomics_Data",
    met_data_name="Ecoli Suaer Dataset",
    met_data_path="../data/sauer_100to300_r200.csv",
    possible_adducts=["[M+F]-", "[M-H]-"],
    mass_tolerance=0.001,
    rt_predictor=None,
    rt_threshold=4.5,
    rt_important_features=None
)
pk.filters.append(metFilter)

# Set up Atomic Composition
atomic_composition_constraints = {"C": [0, 10]}
atomicFilter = AtomicCompositionFilter(atomic_composition_constraints)
pk.filters.append(atomicFilter)

# Run Transformations
pk.transform_all(processes=n_cores, generations=2)

----------------------------------------
Intializing pickaxe object

Done intializing pickaxe object
----------------------------------------



RDKit WARNING: [15:26:24] WARNING: not removing hydrogen atom without neighbors
[15:26:24] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [15:26:24] WARNING: not removing hydrogen atom without neighbors
[15:26:24] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [15:26:26] Can't kekulize mol.  Unkekulized atoms: 2 5 6 7 8 9
[15:26:26] Can't kekulize mol.  Unkekulized atoms: 2 5 6 7 8 9

RDKit ERROR: 
RDKit ERROR: [15:26:26] Can't kekulize mol.  Unkekulized atoms: 2 4 5 6 8 11
[15:26:26] Can't kekulize mol.  Unkekulized atoms: 2 4 5 6 8 11

RDKit ERROR: 
RDKit ERROR: [15:26:26] Can't kekulize mol.  Unkekulized atoms: 1 5 9 10
[15:26:26] Can't kekulize mol.  Unkekulized atoms: 1 5 9 10

RDKit ERROR: 
RDKit ERROR: [15:26:26] Can't kekulize mol.  Unkekulized atoms: 1 5 6 10 11 12
RDKit ERROR: 
[15:26:26] Can't kekulize mol.  Unkekulized atoms: 1 5 6 10 11 12

RDKit ERROR: [15:26:26] Can't kekulize mol.  Unkekulized atoms: 1 5 6 9 10 11
[15:26:26] Can

RDKit ERROR: [15:26:28] Can't kekulize mol.  Unkekulized atoms: 3 5 6 7 8 9
[15:26:28] Can't kekulize mol.  Unkekulized atoms: 3 5 6 7 8 9

RDKit ERROR: 
RDKit ERROR: [15:26:28] Can't kekulize mol.  Unkekulized atoms: 2 4 6 7 8 9 10 11
[15:26:28] Can't kekulize mol.  Unkekulized atoms: 2 4 6 7 8 9 10 11

RDKit ERROR: 
RDKit ERROR: [15:26:28] Can't kekulize mol.  Unkekulized atoms: 3 5 6 7 8 9
[15:26:28] Can't kekulize mol.  Unkekulized atoms: 3 5 6 7 8 9

RDKit ERROR: 


1908 compounds loaded...
(1886 after removing stereochemistry)
----------------------------------------
Filtering Generation 0

Filtering compounds based on match with metabolomics data.
1886 of 1886 compounds selected after Metabolomics filtering of generation 0--took 0.0s.

Done filtering Generation 0
----------------------------------------

----------------------------------------
Filtering Generation 0

Applying filter: Atomic Composition
Filtering Generation 0 with atomic composition {'C': [0, 10]}.
1330 of 1886 compounds remain after applying filter: Atomic Composition--took 0.01s.

Done filtering Generation 0
----------------------------------------

----------------------------------------
Expanding Generation 1

Generation 1: 0 percent complete
Generation 1: 10 percent complete
Generation 1: 20 percent complete
Generation 1: 30 percent complete
Generation 1: 40 percent complete
Generation 1: 50 percent complete
Generation 1: 60 percent complete
Generation 1: 70 percent comple

In [15]:
pk.pickle_pickaxe("../data/ecoli_metab_100t300r200.pk")

# 2. Generate and Apply Filters
Filters can be applied before or after Pickaxe. In this case they are applied afterwards to utilize the full generated network.

The filters are applied in two steps:
1. Feasibility Filter
2. Thermodynamics Filter

## 2.1 Feasibility Filter
The feasibility filter utilizes the DeepRFC code. Here we parallelize the application of this filter and get the feasibility of reactions. If reactions are unable to be predicted, they will also be deemed "infeasible"

In [17]:
rf = ReactionFeasibilityFilter()

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
n_processors = 10

/Users/kevbot/miniconda3/envs/comb/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get Reactions to Check

#### Identify and Analyze "1st Gen" Reactions
This code can be run in jupyter, but is faster outside due to parallelization start method (spawn vs. fork).

Took ~1.5 hours with 12 processes.

#### Identify and Analyze "1st Gen" Reactions

In [ ]:
rxn_1gen_set = set()
for cpd in pk.compounds.values():
    if cpd["Generation"] == 1:
        rxn_1gen_set.update(cpd["Product_of"])

rf_inputs, gen1_failed_reaction = rf._get_inputs(rxn_1gen_set, pk)

In [ ]:
# Chunk data for parallelization
n_processors = 12
inputs = [*rf_inputs.keys()]
n_vals = len(inputs)
chunk_size = n_vals // n_processors + 1 if (add := (n_vals % n_processors) / n_processors) < 1 else ceil(add)
chunked_vals = [*chunks(inputs,  chunk_size)]

rf_input_list = []
for chunk in chunked_vals:
    rf_input_dict = {}
    for r_id in chunk:
        rf_input_dict[r_id] = rf_inputs[r_id]

    rf_input_list.append(rf_input_dict,)

In [ ]:
gen1_feas_results = {}
with mp.Pool(n_processors) as pool:
    i = 0
    for res in pool.imap_unordered(_get_feasibility, rf_input_list):
        print("res", i)
        i += 1
        gen1_feas_results.update(res)

pickle.dump(gen1_feas_results, open("./data/gen1_feas_results.pk", "wb"))

### Get Results Dictionary

In [ ]:
feas_dict_dups = {key: val[2] for key, val in gen1_feas_results.items()}
feas_dict_dups.update({key: "infeasible" for key in gen1_failed_reaction})

# Create dictionary and take feasible if collision and one is feasible
feas_dict = {}
for rxn_id, feas in feas_dict_dups.items():
    rxn_id = rxn_id.split("_")[0]
    if rxn_id not in feas_dict:
        feas_dict[rxn_id] = feas
    elif feas == "feasible":
        feas_dict[rxn_id] = feas

assert len(rxn_1gen_set) == len(feas_dict)

pickle.dump(gen1_feas_results, open(".data/gen1_feas_dict.pk", "wb"))

#### Identify and Analyze "2st Gen" Reactions
This code can be run in jupyter, but is faster outside due to parallelization start method (spawn vs. fork).

Took ~3 hours with 12 processes.

#### Identify and Analyze "1st Gen" Reactions

In [ ]:
rxn_2gen_set = set()
for cpd in pk.compounds.values():
    if cpd["Generation"] == 1:
        rxn_2gen_set.update(cpd["Product_of"])

rf_inputs, gen2_failed_reaction = rf._get_inputs(rxn_2gen_set, pk)

In [ ]:
# Chunk data for parallelization
n_processors = 12
inputs = [*rf_inputs.keys()]
n_vals = len(inputs)
chunk_size = n_vals // n_processors + 1 if (add := (n_vals % n_processors) / n_processors) < 1 else ceil(add)
chunked_vals = [*chunks(inputs,  chunk_size)]

rf_input_list = []
for chunk in chunked_vals:
    rf_input_dict = {}
    for r_id in chunk:
        rf_input_dict[r_id] = rf_inputs[r_id]

    rf_input_list.append(rf_input_dict,)

In [ ]:
gen2_feas_results = {}
with mp.Pool(n_processors) as pool:
    i = 0
    for res in pool.imap_unordered(_get_feasibility, rf_input_list):
        print("res", i)
        i += 1
        gen2_feas_results.update(res)

pickle.dump(gen1_feas_results, open("./data/gen2_feas_results.pk", "wb"))

### Get Results Dictionary

In [ ]:
feas_dict_dups = {key: val[2] for key, val in gen2_feas_results.items()}
feas_dict_dups.update({key: "infeasible" for key in gen2_failed_reaction})

# Create dictionary and take feasible if collision and one is feasible
feas_dict = {}
for rxn_id, feas in feas_dict_dups.items():
    rxn_id = rxn_id.split("_")[0]
    if rxn_id not in feas_dict:
        feas_dict[rxn_id] = feas
    elif feas == "feasible":
        feas_dict[rxn_id] = feas

assert len(rxn_1gen_set) == len(feas_dict)

pickle.dump(gen1_feas_results, open(".data/gen2_feas_dict.pk", "wb"))

## 2.2 Calculate Thermodynamics Feasibility
### Running Externally
- Requires a local [equilibrator cache to run](https://equilibrator.readthedocs.io/en/latest/local_cache.html).
- When using eQuilibrator to calculate the ∆G_rxn the code spawns multiple connections to a SQL server. If using postgres this process is much faster than using the SQLite default.
- As a result, ecoli_rxnfeas_calculator.py is run seperately from the jupyter notebook, and a csv is read in with the results.